In [1]:
!export MSI_SECRET=9v8xXHPAVo3Ra7XFU7dogUTtoVZ3cAdZ
!export MSI_ENDPOINT=http://127.0.0.1:46808/MSI/auth
!env

SHELL=/bin/bash
MSI_SECRET=jRa6OKvwoM4H4Ki1aZ3a8moaCvmnOJc2
CI_LOCAL_UBUNTU_USER=azureuser
DEFAULT_IDENTITY_CLIENT_ID=ce4d7d3c-a425-4dfb-921c-37f0cf5faa46
PWD=/mnt/batch/tasks/shared/LS_root/mounts/clusters/gareth/code
LOGNAME=azureuser
JPY_SESSION_NAME=gsobrien_C_populate_file_Run.ipynb
APPSETTING_WEBSITE_SITE_NAME=AMLComputeInstance
HBI_WORKSPACE=false
HOME=/home/azureuser
MLFLOW_TRACKING_URI=azureml://westeurope.api.azureml.ms/mlflow/v1.0/subscriptions/f5a67d06-2d09-4090-91cc-e3298907a021/resourceGroups/hugo-ml/providers/Microsoft.MachineLearningServices/workspaces/hugo-eessi
LANG=C.UTF-8
PYDEVD_USE_FRAME_EVAL=NO
CLICOLOR=1
INVOCATION_ID=697188f345324b72af0682bc9e9679a5
ENABLE_MLFLOW_AUTOLOG=false
SGX_AESM_ADDR=1
JPY_PARENT_PID=3737
OBO_ENDPOINT=http://127.0.0.1:46808/OBO/token
TERM=xterm-color
CI_NAME=gareth
CI_WORKSPACE=hugo-eessi
USER=azureuser
GIT_PAGER=cat
SHLVL=0
PAGER=cat
AML_CloudName=AzureCloud
MPLBACKEND=module://matplotlib_inline.backend_inline
MKL_THREADING_LAYER=GNU
JOU

In [2]:
from azure.ai.ml import command, MpiDistribution, Input
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azureml.core import Workspace

# authenticate
# authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

credential = DefaultAzureCredential()
print(credential)

In [3]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="f5a67d06-2d09-4090-91cc-e3298907a021",
    resource_group_name="hugo-ml",
    workspace_name="hugo-eessi",
)

print(ml_client)


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fe0a0c2e2c0>,
         subscription_id=f5a67d06-2d09-4090-91cc-e3298907a021,
         resource_group_name=hugo-ml,
         workspace_name=hugo-eessi)


In [4]:
# load in the compute cluster to use
cpu_compute_target="hc44"

cpu_cluster = ml_client.compute.get(cpu_compute_target)
print(f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is.")


You already have a cluster named hc44, we'll reuse it as is.


In [8]:
# populate the file
#from azureml.core import Environment
#from azureml.core.runconfig import DockerConfiguration
#env = Environment(name="devito-docker")
#env.docker.enabled = True
#env.docker.base_image = 'hugoeessi.azurecr.io/devito:6'
#docker_config = DockerConfiguration(use_docker=True)


nprocess=4;

for i in range(0,nprocess):
    
    nproc=" " + str(nprocess)
    jid=" " + str(i)
    script = "gsobrien_B_populate_zarr_file_in_parrallel.py "
    
    # define the command
    command_job = command(
        code="/home/azureuser/cloudfiles/code/",
        command="source /venv/bin/activate; python3 " + script + nproc + jid,
        #command="python3 --version",
        environment= "docker-test1:4",
        # environment=env,
        instance_count=1,
        #max_concurrency_per_instance=4,
        #distribution=MpiDistribution(process_count_per_instance=2),
        compute="hc44",
        display_name="pop_file" + jid,
        #properties = [docker_runtime_config=docker_config]
    )
    returned_job = ml_client.jobs.create_or_update(command_job)
    print(i)
    
print("finished job submission")

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/home/azureuser/cloudfiles/code' 'https://hugoeessi2919381769.blob.core.windows.net/f29181ea-5322-4962-926c-6c1bcf5148f1-5gtn6dtf1do4v97vf03zaal11u/code' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading code (282.85 MBs): 100%|██████████| 282854533/282854533 [00:05<00:00, 51194969.96it/s]




0
1
2
3
finished job submission
